In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import LSTM, Dropout, Dense, BatchNormalization
import keras
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv(r'C:\Users\serez\PycharmProjects\pythonProject1\venv\general_version.FIASCO.csv')
data = data.sort_values('Timestamp')
data 

,Timestamp,Open,High,Low,Close
2398,2022-12-30 22:00:00,16565.77,16570.53,16523.81,16525.37
2397,2022-12-30 23:00:00,16525.91,16571.51,16523.91,16568.69
2396,2022-12-31 00:00:00,16569.36,16677.35,16568.31,16585.06
2395,2022-12-31 01:00:00,16585.06,16597.00,16577.57,16583.27
2394,2022-12-31 02:00:00,16582.81,16618.75,16568.00,16607.48
...,...,...,...,...,...
4,2023-04-09 17:00:00,27905.62,27925.01,27800.00,27866.49
3,2023-04-09 18:00:00,27866.48,27918.26,27835.26,27891.91
2,2023-04-09 19:00:00,27891.91,27942.55,27890.75,27896.98
1,2023-04-09 20:00:00,27896.97,27930.10,27876.68,27906.68


In [8]:
data = data.drop('Timestamp', axis=1)

In [9]:
data

,Open,High,Low,Close
2398,16565.77,16570.53,16523.81,16525.37
2397,16525.91,16571.51,16523.91,16568.69
2396,16569.36,16677.35,16568.31,16585.06
2395,16585.06,16597.00,16577.57,16583.27
2394,16582.81,16618.75,16568.00,16607.48
...,...,...,...,...
4,27905.62,27925.01,27800.00,27866.49
3,27866.48,27918.26,27835.26,27891.91
2,27891.91,27942.55,27890.75,27896.98
1,27896.97,27930.10,27876.68,27906.68


In [11]:
#делим данные
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42, shuffle=False)

In [12]:
#конвертируем в csv
train_data.to_csv('train_FIASCO.csv', index=False)
val_data.to_csv('val_FIASCO.csv', index=False)
test_data.to_csv('test_FIASCO.csv', index=False)

In [15]:
#создаём функцию меток для последующих датафреймов
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i : (i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [18]:
TIME_STEPS = 24

#метки для данных
X_train, y_train = create_sequences(train_data, train_data['Close'], TIME_STEPS)
X_val, y_val = create_sequences(val_data, val_data['Close'], TIME_STEPS)
X_test, y_test = create_sequences(test_data, test_data['Close'], TIME_STEPS)

In [19]:
# код модели
model = Sequential([
    LSTM(units=256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(rate=0.2),
    LSTM(units=256, return_sequences=True),
    Dropout(rate=0.2),
    LSTM(units=128),
    Dropout(rate=0.2),
    Dense(units=64),
    Dropout(rate=0.2),
    Dense(units=1)
])

# компиляция
model.compile(optimizer='adam', loss='mse')

# earlystipping на 5 эпох
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

# обучение
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    shuffle=False
)

# оценка
test_loss = model.evaluate(X_test, y_test)

print(f'Test loss: {test_loss:.2f}')

Epoch 1/50
57/95 [=================>............] - ETA: 3s - loss: 431848352.0000

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

# предсказание на следующий час
next_hour_pred = y_pred[-1][0]
print(f'Предсказание на следующий час: {next_hour_pred:.2f}')
